# Effect of Informal Care on Labor Supply: A Macroeconomic Perspective

## 2018.3.12 ICES, Hosei University

### Daisuke Moriwaki

Cyberagent, Inc., AI Lab        
Keio University, School of Medicine
<daisuke.moriwaki@gmail.com>   
github [daimoriwaki/care_economy](https://github.com/daimoriwaki/care_economy)

In [54]:
#workspace()

In [55]:
using QuantEcon
using Plots
using Interpolations
using Distributions
using DataFrames
srand(20150423);
#using BenchmarkTools
#using ParallelAccelerator

# Motivation
* In 2017, it is reported taht there is 6 million care revievers in Japan by MLHW. The number is expected to increase as the baby boomers, or Dankai generations become "late elderly".
* It is unclear whether the Japanese LTCI system is sustanable in the sense of financially, which has been pointed out by several researchers.
* More importantly, the impact of care needs on economy itself (oppose to accountant's view) should be studied.


* It is easy to predict that "caregiver ratio" (the number of caregivers per one care receiver) explodes in the near future.
* Aside from the financial sustainability, the explosion of caregiver ratio cause several problems
    * insufficient care for elderly (insufficient availability of cheap nursing home, *tokuyou*)
    * Poverty due to care-leave for frail older family
    * *Economy itself* due to less labor supply
* Such a wide range of problem should be tackled by macroeconomic model rather than causal analysis.

# Summary of Results
* I ran a simplified care economy model to see the effect of informal care.
* Informal care has mixed effect on consumption: people consume less with some informal care, but consume more when required informal care is long.
* A comparison of the outcomes from the four policy options indicate fully-covered LTCI raise the labor force participation rate but need the highest tax rate. Introduction of **ceiling** policy is cost effective. 

# Contribution
* possibly the first paper to delineate Japanese LTCI system with heterogeneous agent macromodel
* have some implication for the LTCI policy 

# Literature

* Briefly survery several work in the labor and macro fields.



## Labor supply and informal care (causal studies)

* Active area of research
    * Nishimura and oikawa (2017)
    * Niimi (2017)
    * Oshio and Usu (2016)
    * Yamada and Shimizutani (2015), and me (2015) ....

## Social Security System and Macroeconomy
* Also very active area 
    * Li (EER, 2018) on DI and OAI
    * Kubota (2017) on childcare



## Sustainability due to aging population in Japan
* Kitao (2016, IER)


# Set up *"Care Economy"*

# Demography
* 3 periods: young, middle, and old
    * young: work in the market and consume goods and leisure.Save for the future 22-44
    * middle: work both in the market and home. consume goods and leisure. Save for the future. No mortality risk. 45-64
    * old: no work in both market and home. Those with bad health receive care either in public nursing home or at home. Mortality risk is present. 65+
* in 2015 the three are almost same number.

## Household

* The model economy is populated by many households. 
    * You may imagine either single households without gender and unknown re-production technique or many households with couples and kids where income, assets, health, and time endowment is shared. 
    * More realistic modelling is listed as future study
* wage rate is exogenous to the agent. Given wage rate, labor supply is endogeneous.
* No negative saving is allowed (consider whether we can borrow without collateral).

## Care in detail
* In this economy, every middle-aged households face the risk of becoming care giver.
* In the beginning of the period, the health status of all households is assigned. 
* Then the total *care needs* is determined. 
* The government determine how much care to provide, which is sometimes insufficient. That deliniates reality in Japan (there are 360,000 people on the waiting list).
* Unmet care needs is distributed to potential care givers (*middle*). No state dependency. Imagine that you might become a informal care giver for your uncle, spouse's siblings and so on as well as your own genetic parents and spouse. Unexpectable.
* This scenario totally igonres the positive utility from care-giving. But there are lots of tragedies associated with informal care.

## Preference
* simple and general utility function.
* parameters are set according to Yue (2018)

In [56]:
function calc_utility(c, l,chi = chi, sigma = sigma) 
    if c <= 0 || l <= 0
        return -100000
    else
        return (((c^(1-chi))*(l^chi))^(1-sigma))/(1 - sigma)
    end
end

calc_utility (generic function with 3 methods)

## consumption
* consumption is deterimined by assets, income and saving

In [57]:
function calc_consumption(age, new_asset::Real, income::Real, asset::Real, r::Real=r)
    if age < num_period
        if new_asset >= (1+r)*asset + income
            return 0
        else
            (1 + r)*asset + income - new_asset # allow negative consumption
        end
    else
        return (1 + r)*asset + income
        #return 10000000
    end 
end

calc_consumption (generic function with 2 methods)

## income
* income is determined by hourking hour in market and tax
* retired person collects pension.


In [58]:
function calc_income(wage, hour_mkt, pension, tau)
    (1 - tau)*wage*hour_mkt + pension
end

calc_income (generic function with 1 method)

## pension
* mandatory pension system like real economy.
* the level of benefit is determined by wage history and replacement rate (0.4)

In [59]:
function calc_new_average_income(age, last_average_income, current_wage, current_hour_mkt)
    if age == 1
        return current_wage*current_hour_mkt # calc average income for today
    elseif age >= retirement_age
        return last_average_income # after retired average income is fixed
    else
        return (last_average_income*(age - 1) + current_wage*current_hour_mkt)/age # during working age average income is changed 
    end
end

calc_new_average_income (generic function with 1 method)

In [60]:
function calc_pension(age, average_income, rep_rate = rep_rate, retirement_age = retirement_age)
    if age >= retirement_age
        return rep_rate*average_income
    else
        return 0
    end
end

calc_pension (generic function with 3 methods)

## wage
* wage evolves according to $w_{i,t} = \mu + \rho w_{i, t-1} + \epsilon_{t}$ with $\rho = 0.9$
* use tauchen method to generate transition matrix

## Informal care
* Middle-aged household is assigned care needs randomly.
* The level of care is represented the required hour for care at home.
* The basic assumption is
    * Households are mandated to take care of their parents.
    * They want to use cheap formal care funded by LTCI
    * Due to the resorce constraint, use of formal care is restricted

# Informal care in detail
* Once all the household draw the raw hour at home $h^H$, they are sum up as $\sum_i h^H_i$
* The total demand for care $D^H = \sum_i I (h_i \in (0,  0.1)$
* The unmet care needs $D^{H, unmet} = D^H - NH$ is distributed out to proportionaly to $h^H$
* Numerical excercise is done without empirical studies, which should be done in the future.

In [61]:
function calc_hour_home(age, raw_hour_home, total_raw_hour_home, LTCI_type, NH_rate, ceiling, dep_rate::Real = dep_rate, sick_rate::Real=sick_rate, care_hour::Real=care_hour, num_hh = num_hh)
    if LTCI_type == 1
        if age in middle_age
            share = raw_hour_home/total_raw_hour_home # sahre of care this person take
            return min(share*(1 - NH_rate)*dep_rate*sick_rate*care_hour*num_hh, 0.9999)
        else
            return 0
        end
    else
        if age in middle_age
            return min(raw_hour_home*dep_rate*sick_rate*care_hour, ceiling)
        else
            return 0
        end

    end
end

calc_hour_home (generic function with 5 methods)

## health status
* health status so far only matters how many old need care.
* health is not associated with wage, labor.
* we do not take health into accont explicitly.

## labor supply
* labor supply is endogeneous
* young and middle household optimizes labor supply as well as consumption (saving) for given time endowment
* time endowment is varying across agent due to informal care
* From the first order condition $l = c(1 - \tau)w \frac{\chi}{1 - \chi}$

In [62]:
"calc_hour_mkt(age, leisure, hour_home, retirement_age)

calculate hour in the market.

**Argments**: 

        age, leisure, hour at homt

**Return**

        working hour at market"
function calc_hour_mkt(age::Real, leisure::Real, hour_home::Real, retirement_age::Int=retirement_age)
    if leisure + hour_home >= 1 || age >= retirement_age
        return 0
    else
        return 1 - leisure - hour_home
    end
end

calc_hour_mkt

## Government
* government provide insufficient formal care funded by social security premium contributed by working age, which is resemble to Japanese LTCI
* cost of formal care is determined by personnel expenses of informal care worker.
* The government chooses one of two LTCI system
\begin{eqnarray}
LTCI \, (type 1) &=& w^{care} \phi N^{old} sick rate \, d^H  NH rate \\
LTCI \, (type 2) &=& w^{care} \phi \sum_i(care_i - \bar{care}) \mathbb{1}[care_i > \bar{care}]
\end{eqnarray}
* __type1__ is *proportional sapplements* where the government provide formal care proportional to the level of required care. $w^{care}$ denotes average wage for worker at nurinsg home. $N^{old}$, $sick rate$, $d^H$, $NH rate$ represents the population of old, the rate of care receivers to the old and required care hour per sick old. Finally, $\phi$ is a multiplier ly inside $(0, 1)$. This multiplier represents *efficiency* of nurinsg home.   
* __type2__ is *ceiling*. The care above the ceiling is given by the government.
* $\phi:$ Usually, typical informal care giver provide care to one care receiver, whereas (formal) care worker can work for multiple care receivers. They also benefit from the division of labor.

* On the other hand, the government collect $\tau\sum_i w_i h_{M, i}$ from all the young and middle.

# Quantitative Exercise

## Solution
* The model is solved (ideally) by method of simulated moment.
* So far no calibration made
* Optimization by households is done by backward induction.

## Value function
* value function optimization $v_t = max_{c_t, l_t} (u(c_t, l_t) + \beta \mathbb{E}\left( v_{t+1}(a_{t+1}) \right))$ 
* wrt usual budget constraint with no borrowing


In [63]:
"value_function(utility, expected_value, beta)
calculate instant value for given utility and expected values in the next period"
function value_function(utility::Real, expected_value::Real, beta::Real = beta)
    utility + beta*expected_value
end

value_function

## Expected values
* expected values are dot product of transition matrix and state-dependent values 

In [64]:
function interpolate_vector(current_value::Real, grid, trans_mat, num_grid = num_grid)
    itp = interpolate((grid, grid), trans_mat, Gridded(Linear()))
    vec = itp[current_value, grid]
    for i in 1:num_grid
        if vec[i] < 0
            vec[i] = 0
        end
    end
    return vec
end

interpolate_vector (generic function with 2 methods)

In [65]:
"gen_prob_vec
return probability vector for arbitrary wage, hour at home, current_average_income

**Args**
current_wage
current_hour_home
current_average_wage

**return**
vector of probability
"
function gen_prob_vec(current_wage, current_hour_home, wage_grid = wage_grid, hour_home_grid = hour_home_grid,
        wage_trans_mat = wage_trans_mat, hour_home_trans_mat= hour_home_trans_mat)
    wage_prob_vector = interpolate_vector(current_wage, wage_grid, wage_trans_mat)
    hour_home_prob_vector = interpolate_vector(current_hour_home, hour_home_grid, hour_home_trans_mat)
    return kron(wage_prob_vector, hour_home_prob_vector)
end

gen_prob_vec

In [66]:
"get_new_value(new_asset, max_value_mat, asset_grid = asset_grid, num_grid = num_grid): 


get vector of maximized value for next period given choice asset and states current wage, current hour at home. 
max_value_mat should be asset_grid x wage, relative_hour_home"
function get_new_value(new_asset, new_average_income, max_value_mat, asset_grid = asset_grid, wage_grid = wage_grid,  num_grid = num_grid)
    v_new = zeros(num_grid^2, 1)
    for i in 1:num_grid^2
        temp_slice = max_value_mat[:,i]
        temp_mat= reshape(temp_slice, num_grid, num_grid)
        knots = (asset_grid, wage_grid)        
        itp = interpolate(knots, temp_mat, Gridded(Linear()))
        v_new[i] = itp[new_asset, new_average_income]
    end
    return v_new
end

get_new_value

## Generate Policy Function

In [67]:
"generate_max_values_matrix(value_function, asset_grid, leisure_grid, wage_grid, hour_home_grid, num_grid):

maximize value with respect to leisure and consumption
"

function generate_max_values_matrix(age, tau, LTCI_type, NH_rate, ceiling, value_function::Function, new_max_value_mat = 
        zeros(num_grid, num_grid^3), asset_grid = asset_grid, leisure_grid = leisure_grid,
        wage_grid = wage_grid, hour_home_grid = hour_home_grid, num_grid = num_grid, total_raw_hour_home = total_raw_hour_home)
    # initializing matrices
    value_matrix = zeros(num_grid^2, num_grid^4);
    consumption_mat = zeros(num_grid^2, num_grid^4);
    leisure_mat = zeros(num_grid^2, num_grid^4);
    income_mat = zeros(num_grid^2, num_grid^4);
    new_average_income_mat = zeros(num_grid^2, num_grid^4);
    new_asset_mat = ones(num_grid^2, num_grid^4);
    wage_mat = zeros(num_grid^2, num_grid^4);
    new_asset_mat = zeros(num_grid^2, num_grid^4);
    hour_home_mat = zeros(num_grid^2, num_grid^4); # hour home (realized)
    raw_hour_home_mat = zeros(num_grid^2, num_grid^4); # relative hour home (not real one)
    hour_mkt_mat = zeros(num_grid^2, num_grid^4);

    for new_asset_ind = 1:num_grid  
        for leisure_ind = 1:num_grid
            for asset_ind = 1:num_grid
                for average_income_ind = 1:num_grid
                    for wage_ind = 1:num_grid
                        for hour_home_ind = 1:num_grid
                            new_asset = asset_grid[new_asset_ind]      
                            leisure = leisure_grid[leisure_ind]
                            asset = asset_grid[asset_ind]
                            wage = wage_grid[wage_ind] # wage is 0 when retired
                            #if age < retirement_age
                            #    wage = wage_grid[wage_ind] # wage is 0 when retired
                            #else
                            #    wage = 0
                            #end
                            raw_hour_home = hour_home_grid[hour_home_ind]
                            hour_home = calc_hour_home(age, raw_hour_home, total_raw_hour_home, LTCI_type, NH_rate, ceiling)
                            hour_mkt = calc_hour_mkt(age, leisure, hour_home)
                            if age == 1
                                average_income = 0
                            else
                                average_income = wage_grid[average_income_ind]
                            end
                            new_average_income = calc_new_average_income(age, average_income, wage,  hour_mkt) # calculate average income

                            pension = calc_pension(age, average_income)
                            income = calc_income(wage, hour_mkt, pension, tau)
                            consumption = calc_consumption(age, new_asset, income, asset)
                            utility = calc_utility(consumption, leisure)
                            
                            row_ind = new_asset_ind + (leisure_ind - 1)*num_grid
                            col_ind = asset_ind + (average_income_ind - 1)*num_grid+ (wage_ind - 1)*num_grid^2 + (hour_home_ind - 1)*num_grid^3
                            
                            # store variables
                            consumption_mat[row_ind, col_ind] = consumption;
                            new_asset_mat[row_ind, col_ind] = new_asset;
                            leisure_mat[row_ind, col_ind] = leisure;
                            income_mat[row_ind, col_ind] = income;
                            wage_mat[row_ind, col_ind] = wage;
                            new_average_income_mat[row_ind, col_ind] = new_average_income;
                            raw_hour_home_mat[row_ind, col_ind] = raw_hour_home;
                            hour_home_mat[row_ind, col_ind] = hour_home;
                            hour_mkt_mat[row_ind, col_ind] = hour_mkt;
                            
                            # calc expected_value
                            v_new_vec = get_new_value(new_asset, new_average_income, new_max_value_mat)
                            p = gen_prob_vec(wage, hour_home)
                            expected_value = dot(v_new_vec, p)
                            value_matrix[row_ind, col_ind] = value_function(utility, expected_value)
                        end
                    end
                end
            end
        end
    end
    # get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
    #max_value = maximum(value_matrix, 1)
    max_value, max_ind = findmax(value_matrix, 1)
    # reshape value matrix to (asset_grid x new_average_income)  x (w x h_H)
    # this should be fix throughout the simulation
    max_values_reshaped = reshape(max_value, num_grid^2, num_grid^2);
    return max_values_reshaped, max_ind, new_asset_mat, consumption_mat, income_mat, wage_mat, hour_home_mat, raw_hour_home_mat, 
    hour_mkt_mat, new_average_income_mat
end



generate_max_values_matrix

In [68]:
function gen_policy_function(tau, LTCI_type, NH_rate, ceiling, num_grid = num_grid, value_function = value_function)
    
    new_max_value_matrix = zeros(num_grid^2, num_grid^2);
    pol_consumption = zeros(num_period, num_grid^4);
    pol_new_asset = zeros(num_period, num_grid^4);
    pol_hour_mkt = zeros(num_period, num_grid^4);
    pol_income = zeros(num_period, num_grid^4);
    pol_new_average_income = zeros(num_period, num_grid^4);

    for t in (3,2,1)
        max_values_reshaped, max_ind, new_asset_mat, consumption_mat, income_mat, wage_mat, hour_home_mat, raw_hour_home_mat, 
        hour_mkt_mat, new_average_income_mat = 
        generate_max_values_matrix(t, tau, LTCI_type, NH_rate, ceiling, value_function, new_max_value_matrix)
        
        pol_consumption[t,:] = consumption_mat[max_ind]
        pol_hour_mkt[t,:] = max(hour_mkt_mat[max_ind], 0)
        pol_income[t,:] = income_mat[max_ind]
        pol_new_average_income[t,:] = new_average_income_mat[max_ind]
        pol_new_asset[t,:] = max(new_asset_mat[max_ind], 0)
        new_max_value_matrix = max_values_reshaped
    end
        return pol_consumption, pol_hour_mkt, pol_income, pol_new_asset, pol_new_average_income
end

gen_policy_function (generic function with 3 methods)

# Simulation

## Setting
* we simulate 100,000 households with 3 periods
* initial wage is randomly distributed, assets are set to 0 for young.
* wage evolves according to the distribution. Highly persistent with $\rho = 0.9$ 
* hour at home are set randomly

In [69]:
# setting

### Economy ###
const num_threads = 16;
const r = 0.01; # interest rate
const dep_rate = 1 # dependency rate
const wage_care = 0.7; # care workers' wage rate. ratio to all industry
const sick_rate = 0.3; # need care old / old
const care_hour = 0.3; # care demand per care receiver in hours
const middle_age = (2); # when giving care

### Government ###
#tau = 0.3; # income tax rate
const rep_rate = 0.3; # replacement rate
#NH_rate = 0.5; # government formal care provision rate. ratio to needed care
const retirement_age = 3; # age when household can take up pension
const num_period = 3; # number of periods when hh dies surely
const phi = 2/3 # cost efficiency of nursing home compared tp informal care 
# LTCI_type = 1; # 1 = proportionaly distribution, 2 = ceiling
#ceiling = 0.2; # ceiling of informal care. above that the government take care of the needy


### Preference ###
const beta = 0.9; # discount rate
const chi = .28; # share of leisure in utilty
const sigma = 4; # intertemporal substitution

### numerical excersize ###
num_grid = 9; # number of grid 9
const min_asset = 0; # minimum asset
const max_asset = 10; # maximum asset
const min_leisure = 0.001;
const max_leisure = 0.999;

# wage trainsition matrix
# "const" make it faster 
const wage_rho = 0.9 # param for wage function
const wage_sigma = 0.2# param for wage function standard deviation for epsilon
const wage_mu = 0 # mean wage
const wage_n_std = 3  # length of grid

const wage_markovchain = tauchen(num_grid, wage_rho, wage_sigma, wage_mu, wage_n_std);
const wage_trans_mat = wage_markovchain.p;
const wage_grid = exp(wage_markovchain.state_values);

# informal care grid and transition matrix

hour_home_rho = 0 # param for wage function
hour_home_sigma = 1# param for wage function standard deviation for epsilon
hour_home_mu = 0 # mean hour at home
hour_home_n_std = 3 # length of grid

hour_home_markovchain = tauchen(num_grid, hour_home_rho, hour_home_sigma, hour_home_mu, hour_home_n_std);
hour_home_trans_mat = hour_home_markovchain.p;
hour_home_grid = exp(hour_home_markovchain.state_values);

asset_grid = linspace(min_asset, max_asset, num_grid);
leisure_grid = linspace(min_leisure, max_leisure, num_grid);

# simulation 100 thousand H/H
# initializing matrices
num_hh = 100000; # 100000

asset_record = zeros(num_hh, num_period);
wage_record = zeros(num_hh, num_period);
average_income_record = zeros(num_hh,num_period);
raw_hour_home_record = zeros(num_hh, num_period);
hour_home_record = zeros(num_hh, num_period);
hour_mkt_record = zeros(num_hh, num_period);
consumption_record = zeros(num_hh, num_period);

# wage is first generated by normal and transformed into exponentiated value
wage_dist = Normal(wage_mu, wage_sigma^2/(1 - wage_rho^2));
log_wage = rand(wage_dist, num_hh, 1);
wage_record[:,1] = exp(log_wage);
eps_dist = Normal(0, wage_sigma);
wage_record[:,2] = exp(wage_rho*log_wage + rand(eps_dist, num_hh, 1));

# raw hour home. the hour ac
hour_home_dist = Normal(hour_home_mu, hour_home_sigma^2);
log_hour_home = rand(hour_home_dist, num_hh, num_period);
raw_hour_home_record = exp(log_hour_home);
raw_hour_home_record[:, 1] = 0;
total_raw_hour_home = sum(raw_hour_home_record);

In [70]:
function simulator(LTCI_type, NH_rate, ceiling, pol_consumption, pol_new_asset, pol_hour_mkt, pol_new_average_income, consumption_record = consumption_record,
        asset_record = asset_record, hour_mkt_record = hour_mkt_record, raw_hour_home_record = raw_hour_home_record, 
        average_income_record = average_income_record, asset_grid = asset_grid, hour_home_grid = hour_home_grid,  
        wage_grid = wage_grid, num_grid = num_grid, retirement_age = retirement_age,  num_period = num_period)
    
    

    for t = 1:num_period
        pol_consumption_reshape = reshape(pol_consumption[t,:], num_grid, num_grid, num_grid, num_grid) # reshape for interpolate    
        pol_new_asset_reshape = reshape(pol_new_asset[t,:], num_grid, num_grid, num_grid, num_grid) # reshape for interpolate    
        pol_hour_mkt_reshape = reshape(pol_hour_mkt[t,:], num_grid, num_grid, num_grid, num_grid) # reshape for interpolate    
        pol_new_average_income_reshape = reshape(pol_new_average_income[t,:], num_grid, num_grid, num_grid, num_grid) # reshape for interpolate    
        
        consumption_itp = interpolate((asset_grid, wage_grid, wage_grid, hour_home_grid), pol_consumption_reshape, Gridded(Linear()));
        new_asset_itp = interpolate((asset_grid, wage_grid, wage_grid, hour_home_grid), pol_new_asset_reshape, Gridded(Linear()));
        hour_mkt_itp = interpolate((asset_grid, wage_grid, wage_grid, hour_home_grid), pol_hour_mkt_reshape, Gridded(Linear()));
        new_average_income_itp = interpolate((asset_grid, wage_grid, wage_grid, hour_home_grid), pol_new_average_income_reshape, Gridded(Linear()));


        for n = 1:num_hh
            if t < retirement_age
                asset_record[n,t+1] = new_asset_itp[asset_record[n,t], average_income_record[n,t], wage_record[n,t],
                    raw_hour_home_record[n,t]]
                average_income_record[n,t + 1] = new_average_income_itp[asset_record[n,t], average_income_record[n,t], 
                    wage_record[n,t], raw_hour_home_record[n,t]]
            end
            hour_mkt_record[n,t] = hour_mkt_itp[asset_record[n,t], average_income_record[n,t], wage_record[n,t], raw_hour_home_record[n,t]]
            hour_home_record[n,t] = calc_hour_home(t, raw_hour_home_record[n,t], total_raw_hour_home, LTCI_type, NH_rate, ceiling);
            consumption_record[n,t] = consumption_itp[asset_record[n,t], average_income_record[n,t], wage_record[n,t], raw_hour_home_record[n,t]]

            
            
        end
    end
    return consumption_record, asset_record, hour_mkt_record, hour_home_record, average_income_record
end

simulator (generic function with 12 methods)

# Fiscal sustainability of LTCI

In [71]:
# cost 
function LTCI_cost(LTCI_type, NH_rate, ceiling, raw_hour_home_record = raw_hour_home_record, hour_home_record = hour_home_record, 
        phi = phi, wage_care = wage_care, care_hour = care_hour,sick_rate = sick_rate, 
        dep_rate = dep_rate, total_raw_hour_home = total_raw_hour_home)
    if LTCI_type == 1
        if NH_rate <= 0 
            return 0
        else
            return NH_rate*phi*wage_care*num_hh*dep_rate*sick_rate*care_hour
        end
    end
    if LTCI_type == 2
        if ceiling <= 0
            return 0
        else
            temp_care_hour = zeros(num_hh, 1)
            for i in 1:num_hh                
                temp_care_hour[i] = raw_hour_home_record[i]*dep_rate*care_hour*sick_rate*num_hh/total_raw_hour_home
            end
            return sum((temp_care_hour - ceiling).*(temp_care_hour .>= ceiling))
        end
    end
end

LTCI_cost (generic function with 9 methods)

In [72]:
function calc_revenue(wage_record, hour_mkt_record, tau)
    sum(wage_record.*hour_mkt_record*tau)
end

calc_revenue (generic function with 1 method)

In [73]:
function calc_pension_cost(average_income_record, rep_rate = rep_rate,  num_period = num_period, 
        retirement_age = retirement_age)
    sum(average_income_record[:,3])*rep_rate
end

calc_pension_cost (generic function with 4 methods)

# iteration
* set  $\tau$, LTCI type, NH rate, ceiling, replacement rate and see if converge

In [74]:
while nworkers() < num_threads
    addprocs(1);
end

In [75]:
function iteration(tau, LTCI_type, NH_rate, ceiling, tol = 0.05, lr = 0.01, drep_rate = rep_rate, num_grid = num_grid, 
        raw_hour_home_record = raw_hour_home_record, wage_record = wage_record, value_function = value_function, ratio = 0)
    while abs(ratio - 1) > tol
        
        pol_consumption, pol_hour_mkt, pol_income, pol_new_asset, pol_new_average_income = gen_policy_function(tau, LTCI_type, NH_rate, ceiling, num_grid, value_function)
        consumption_record, asset_record, hour_mkt_record, hour_home_record, new_average_income_record = simulator(LTCI_type, NH_rate, ceiling, pol_consumption, pol_new_asset, pol_hour_mkt, pol_new_average_income)

        cost = LTCI_cost(LTCI_type, NH_rate, ceiling, raw_hour_home_record) + calc_pension_cost(average_income_record, rep_rate)
        revenue = calc_revenue(wage_record, hour_mkt_record, tau)
        ratio = cost/revenue
        if ratio > 1 + tol
            tau = tau + lr
        elseif ratio < 1 - tol
            tau = tau - lr
        end
        println("tax rate is $tau and ratio is $ratio cost is $cost revenue is $revenue")
    end
    return tau, LTCI_type, NH_rate, ceiling, consumption_record, asset_record, hour_mkt_record, hour_home_record
end

iteration (generic function with 9 methods)

# Exercise

## benchmark economy
* LTCI is provided proportional to the level of care. Half of the care is covered by government funded formal care.
* Other policy
    * Pension is proportional to their lifetime income. Almost same as **Kosei Nenkin** or employees' pension insurance.
    * Balanced budget, which is unlikely in the real world.
    * Flat tax with rate consistent with the spending by numerical iteration.

In [76]:
tau = 0.249;
LTCI_type = 1;
NH_rate = 0.5;
ceiling = NaN;
tol = 0.05;
@time iter_results = iteration(tau, LTCI_type, NH_rate, ceiling, tol);

tax rate is 0.239 and ratio is 0.6554005856663571 cost is 26895.702664624176 revenue is 41037.04398933188
tax rate is 0.22899999999999998 and ratio is 0.682823204313485 cost is 26895.702664624176 revenue is 39388.96993353542
tax rate is 0.21899999999999997 and ratio is 0.7126408114887465 cost is 26895.702664624176 revenue is 37740.895877738956
tax rate is 0.20899999999999996 and ratio is 0.7437183310142131 cost is 27472.06321290302 revenue is 36938.80070891786
tax rate is 0.19899999999999995 and ratio is 0.7773649912789936 cost is 28238.40905699058 revenue is 36325.80496136069
tax rate is 0.18899999999999995 and ratio is 0.8161596037072084 cost is 28305.672461110946 revenue is 34681.54063560515
tax rate is 0.17899999999999994 and ratio is 0.8592687371077978 cost is 28323.58115904915 revenue is 32962.42483391534
tax rate is 0.16899999999999993 and ratio is 0.9072725771696861 cost is 28323.58115904915 revenue is 31218.381191909237
tax rate is 0.16899999999999993 and ratio is 0.9609573450

In [77]:
tau = iter_results[1];
#LTCI_type = iter_results[2];
#NH_rate = iter_results[3];
#ceiling = iter_results[4];
consumption_record = iter_results[5];
asset_record = iter_results[6];
hour_mkt_record = iter_results[7];
hour_home_record = iter_results[8];

## impact of informal care on lifetime consumption

In [78]:
histogram(hour_home_record[:, 2], title = "Hour at home (Middle)", nbins = 20, ylabel = "num of households", xlabel = "hour at home")

In [79]:
# hour at home vs coonsumption
lifetime_cons = sum(consumption_record, 2);
scatter(hour_home_record[1:1000,2], lifetime_cons[1:1000], title = "hour at home vs lifetime consumption", xlab = "hour at home", ylab = "lifetime consumption")

In [80]:
# wage vs coonsumption
mean_wage = mean(wage_record[:,1:2], 2);
scatter(mean_wage[1:1000], lifetime_cons[1:1000], title = "wage rate vs lifetime consumption", xlab = "wage rate", ylab = "lifetime consumption")

# inequality

In [81]:
# percentile by hour at home
dat = hcat(hour_home_record[:,2], lifetime_cons);
dat = sortrows(dat);
ptiles = (.01, .25, .5, .75, .99);
p_vec = zeros(Int, size(ptiles, 1));
# find row index in the array for each percentilep_vec = zeros(Int, size(ptiles, 1)); 
for (ind, p) in enumerate(ptiles)
    tmp = trunc(p*num_hh)
    p_vec[ind] = Int.(tmp) # convert to Int
end

In [82]:
p_vec

5-element Array{Int64,1}:
  1000
 25000
 50000
 75000
 99000

In [83]:
# percentiles
bar(p_vec/num_hh*100, dat[p_vec, 2], xlim = (-30, 130), title = "lifetime cons by hour at home", 
xlabel = "percentile of hour at home", ylabel = "lifetime consumption")

In [84]:
# percentile by wage
dat = hcat(mean_wage, lifetime_cons);
dat = sortrows(dat);

for (ind, p) in enumerate(ptiles)
    tmp = trunc(p*num_hh) + 1
    p_vec[ind] = Int.(tmp) # convert to Int
end

In [85]:
# percentiles
bar(p_vec/num_hh*100, dat[p_vec, 2], xlim = (-30, 130), title = "lifetime cons by wage", 
xlabel = "percentile of wage", ylabel = "lifetime consumption")

## Policy and Economy
* compare output and labor supply when policy is changed
* benchmark
    * LTCI is proportional with rate = 50%
* fully covered
    * LTCI is proportional with rate = 100%
* laissez-faire
    * no LTCI at all
* ceiling 
    * LTCI takes care of all the serious care receivers with ceiling = 0.2. no one needs to spend more than 20 % of their time to care-giving
    
   

In [86]:
# saving results for benchmark economy
results = DataFrame(name = "benchmark", LTCI_type = "proportional", NH_rate = 0.5, ceiling = "", 
    tau = tau, gdp = sum(wage_record.*hour_mkt_record), labor_supply=sum(hour_mkt_record))

,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,benchmark,proportional,0.5,,0.16899999999999993,174404.36420061032,168243.92706045182


In [87]:
# fully covered economy
tau = 0.3;
LTCI_type = 1;
NH_rate = .9999;
ceiling = NaN;
tol = 0.05;
@time iter_results = iteration(tau, LTCI_type, NH_rate, ceiling, tol);
tau = iter_results[1];
LTCI_type = iter_results[2];
NH_rate = iter_results[3];
ceiling = iter_results[4];
consumption_record = iter_results[5];
asset_record = iter_results[6];
hour_mkt_record = iter_results[7];
hour_home_record = iter_results[8];
results_fully = DataFrame(name = "fully covered", LTCI_type = "proportional", NH_rate = NH_rate, ceiling = "", 
    tau = tau, gdp = sum(wage_record.*hour_mkt_record), labor_supply=sum(hour_mkt_record))

tax rate is 0.29 and ratio is 0.5891941332475366 cost is 28382.884689633313 revenue is 48172.381712614384
tax rate is 0.27999999999999997 and ratio is 0.6095111723250378 cost is 28382.884689633313 revenue is 46566.635655527236
tax rate is 0.26999999999999996 and ratio is 0.6273614955334436 cost is 29236.307468138228 revenue is 46602.01124278226
tax rate is 0.25999999999999995 and ratio is 0.6505971064791267 cost is 29236.307468138228 revenue is 44937.653698397175
tax rate is 0.24999999999999994 and ratio is 0.6756200721129392 cost is 29236.307468138228 revenue is 43273.296154012096
tax rate is 0.23999999999999994 and ratio is 0.7026448749974568 cost is 29236.307468138228 revenue is 41608.93860962702
tax rate is 0.22999999999999993 and ratio is 0.7319217665898116 cost is 29236.286741505828 revenue is 39944.551557366955
tax rate is 0.21999999999999992 and ratio is 0.7637444520937163 cost is 29236.286741505828 revenue is 38280.195242476664
tax rate is 0.2099999999999999 and ratio is 0.791

,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,fully covered,proportional,0.9999,,0.17999999999999988,176735.94461930104,170485.29822204277


In [88]:
results = vcat(results, results_fully)

,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,benchmark,proportional,0.5,,0.16899999999999993,174404.36420061032,168243.92706045182
2,fully covered,proportional,0.9999,,0.17999999999999988,176735.94461930104,170485.29822204277


In [89]:
#laissez-faire
tau = 0.159;
LTCI_type = 1;
NH_rate = 0.00001;
ceiling = 0;
tol = 0.05;
@time iter_results = iteration(tau, LTCI_type, NH_rate, ceiling, tol);
tau = iter_results[1];
LTCI_type = iter_results[2];
NH_rate = iter_results[3];
ceiling = iter_results[4];
consumption_record = iter_results[5];
asset_record = iter_results[6];
hour_mkt_record = iter_results[7];
hour_home_record = iter_results[8];
results_lf = DataFrame(name = "laissez-faire", LTCI_type = "proportional", NH_rate = NH_rate, ceiling = "", 
    tau = tau, gdp = sum(wage_record.*hour_mkt_record), labor_supply=sum(hour_mkt_record))


tax rate is 0.149 and ratio is 0.9456782818584664 cost is 25876.772350456307 revenue is 27363.187721307022
tax rate is 0.149 and ratio is 1.009146622922793 cost is 25876.772350456307 revenue is 25642.232518709097
249.313697 seconds (3.01 G allocations: 342.773 GB, 24.62% gc time)


,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,laissez-faire,proportional,1.0e-5,,0.149,172095.5202597926,166029.53423345048


In [90]:
results = vcat(results, results_lf)

,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,benchmark,proportional,0.5,,0.16899999999999993,174404.36420061032,168243.92706045182
2,fully covered,proportional,0.9999,,0.17999999999999988,176735.94461930104,170485.29822204277
3,laissez-faire,proportional,1.0e-5,,0.149,172095.5202597926,166029.53423345048


In [91]:
#ceiling
#laissez-faire
tau = 0.159;
LTCI_type = 2;
NH_rate = NaN;
ceiling = 0.1;
tol = 0.05;
@time iter_results = iteration(tau, LTCI_type, NH_rate, ceiling, tol);
tau = iter_results[1];
LTCI_type = iter_results[2];
NH_rate = iter_results[3];
ceiling = iter_results[4];
consumption_record = iter_results[5];
asset_record = iter_results[6];
hour_mkt_record = iter_results[7];
hour_home_record = iter_results[8];
results_ceiling = DataFrame(name = "ceiling", LTCI_type = "ceiling", NH_rate = "", ceiling = ceiling, 
    tau = tau, gdp = sum(wage_record.*hour_mkt_record), labor_supply=sum(hour_mkt_record))

tax rate is 0.149 and ratio is 0.9457381848536902 cost is 25452.041502080538 revenue is 26912.35471899454
tax rate is 0.149 and ratio is 1.0092105462532666 cost is 25452.041502080538 revenue is 25219.753793271615
246.340825 seconds (3.55 G allocations: 342.864 GB, 24.36% gc time)


,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,ceiling,ceiling,,0.1,0.149,169260.09257229272,163297.3185822614


In [92]:
results = vcat(results, results_ceiling)

,name,LTCI_type,NH_rate,ceiling,tau,gdp,labor_supply
1,benchmark,proportional,0.5,,0.16899999999999993,174404.36420061032,168243.92706045182
2,fully covered,proportional,0.9999,,0.17999999999999988,176735.94461930104,170485.29822204277
3,laissez-faire,proportional,1.0e-5,,0.149,172095.5202597926,166029.53423345048
4,ceiling,ceiling,,0.1,0.149,169260.09257229272,163297.3185822614


In [93]:
bar(results[:name], results[:tau], title = "tax rate")



In [94]:
bar(results[:name], results[:gdp], title = "gdp")

In [95]:
bar(results[:name], results[:gdp], title = "labor supplu")

## Lots to be done